In [2]:
import os
import json
from PIL import Image

import os
import json
from PIL import Image

def pad_image_and_update_labels(image_path, label_path, output_image_path, output_label_path, desired_height=1024):
    # Open the image
    image = Image.open(image_path)
    width, height = image.size

    # Read the label data
    with open(label_path, 'r') as label_file:
        labels = json.load(label_file)

    if height < desired_height:
        # Calculate the number of pixels needed to pad to 1024
        top_padding = desired_height - height

        # Create a new image with the desired size and white background
        new_image = Image.new("RGB", (width, desired_height), (255, 255, 255))

        # Paste the original image onto the new image, at the bottom
        new_image.paste(image, (0, top_padding))

        # Update the label Y-values with the top padding
        for label in labels:
            label['y'] += top_padding

    else:
        # If the image is taller than the desired height, crop from the top
        crop_y_start = height - desired_height
        new_image = image.crop((0, crop_y_start, width, height))

        # Update the label Y-values by subtracting the crop start Y value
        for label in labels:
            # Make sure the label 'y' doesn't go into negative after the crop
            label['y'] = max(label['y'] - crop_y_start, 0)

    # Save the new image and updated labels
    new_image.save(output_image_path)
    with open(output_label_path, 'w') as label_file:
        json.dump(labels, label_file, indent=4)

    print(f"Image and label for {image_path} processed and saved to {output_image_path} and {output_label_path}")


# Directory containing images and labels
image_directory = "./TMA_WSI_PNGs"
label_directory = "TMA_WSI_Labels_old"
output_image_directory = "./TMA_WSI_Padded_PNGs"
output_label_directory = "./TMA_WSI_Labels_updated"

# Make sure the output directories exist
os.makedirs(output_image_directory, exist_ok=True)
os.makedirs(output_label_directory, exist_ok=True)

# Iterate over all files in the directory
for filename in os.listdir(image_directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(image_directory, filename)
        label_path = os.path.join(label_directory, os.path.splitext(filename)[0] + '.json')
        output_image_path = os.path.join(output_image_directory, filename)
        output_label_path = os.path.join(output_label_directory, os.path.splitext(filename)[0] + '.json')

        if os.path.exists(label_path):
            pad_image_and_update_labels(image_path, label_path, output_image_path, output_label_path)
        else:
            print(f"No label found for {filename}, skipping.")


Image and label for ./TMA_WSI_PNGs/ABC_106_1_010_1.png processed and saved to ./TMA_WSI_Padded_PNGs/ABC_106_1_010_1.png and ./TMA_WSI_Labels_updated/ABC_106_1_010_1.json
Image and label for ./TMA_WSI_PNGs/ABC_099_1_009_1.png processed and saved to ./TMA_WSI_Padded_PNGs/ABC_099_1_009_1.png and ./TMA_WSI_Labels_updated/ABC_099_1_009_1.json
Image and label for ./TMA_WSI_PNGs/158868.png processed and saved to ./TMA_WSI_Padded_PNGs/158868.png and ./TMA_WSI_Labels_updated/158868.json
Image ./TMA_WSI_PNGs/ABC_103_1_009_1.png is already 1198px in height, no padding or label update needed.
Image and label for ./TMA_WSI_PNGs/158869.png processed and saved to ./TMA_WSI_Padded_PNGs/158869.png and ./TMA_WSI_Labels_updated/158869.json
Image and label for ./TMA_WSI_PNGs/ABC_096_1_009_1.png processed and saved to ./TMA_WSI_Padded_PNGs/ABC_096_1_009_1.png and ./TMA_WSI_Labels_updated/ABC_096_1_009_1.json
Image and label for ./TMA_WSI_PNGs/ABC_109_1_010_1.png processed and saved to ./TMA_WSI_Padded_PNGs